In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2


2023-11-27 17:14:13.986486: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
interpreter = tf.lite.Interpreter(model_path="../models/3.tflite")
interpreter.allocate_tensors()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
def draw_key_points(frame, keypoints, conf_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    i = 0
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > conf_threshold:
            cv2.circle(frame,(int(kx), int(ky)), 1, (0, 255, 0), 5)
            cv2.circle(frame,(int(10), int(10)), 2, (0, 255, 0), 5)
        if i == 0:
            print(ky, ky/720, kx, kx/1280)
        i += 1
    return frame


In [28]:
frame.shape


(720, 1280, 3)

In [5]:
cap = cv2.VideoCapture(0)
dynamic_variable = 0
while cap.isOpened():
    ret, frame = cap.read()

    img = frame.copy()
    img = cv2.resize(img, (192, 192))
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192, 192)
    input_image = tf.cast(img, dtype=tf.float32)

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]["index"], np.array(input_image))
    interpreter.invoke()
    key_points_with_scores = interpreter.get_tensor(output_details[0]["index"])

    draw_key_points(frame, key_points_with_scores, conf_threshold=0.5)

    # Define the text to be displayed
    dynamic_variable += 1

    text = f"Variable Value: {dynamic_variable}"

    # Coordinates where the text will appear
    text_position = (50, 50)

    # Font settings
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_color = (255, 255, 255)  # White color
    line_type = 2

    # Put the text on the frame
    cv2.putText(frame, text, text_position, font, font_scale, font_color, line_type)

    cv2.imshow("Live Feed Test", frame)

    if cv2.waitKey(10) & 0xFF==ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


445.04340648651123 0.6181158423423767 685.3314208984375 0.5354151725769043
441.2308073043823 0.612820565700531 689.4014739990234 0.5385949015617371
445.76271057128906 0.619114875793457 685.3919219970703 0.5354624390602112
451.6325855255127 0.6272674798965454 690.6668090820312 0.5395834445953369
450.5931758880615 0.6258238554000854 688.3712768554688 0.537790060043335
452.90536880493164 0.629035234451294 687.442626953125 0.5370645523071289
442.1620273590088 0.6141139268875122 689.9214172363281 0.5390011072158813
444.4541358947754 0.6172974109649658 685.4851531982422 0.5355352759361267
441.5360641479492 0.6132445335388184 691.7000579833984 0.54039067029953
441.514949798584 0.6132152080535889 687.9420471191406 0.5374547243118286
442.8640365600586 0.615088939666748 689.5733642578125 0.538729190826416
443.6747074127197 0.6162148714065552 689.9617004394531 0.5390325784683228
445.8264398574829 0.6192033886909485 684.9357604980469 0.5351060628890991
446.26134395599365 0.6198074221611023 683.872

-1